In [37]:
import os

cwd = os.getcwd()

In [38]:
import pandas as pd


df = pd.read_csv(cwd + '/data/posts_clean.csv')

In [39]:
df.head()

,Post Link,Title,Body,Tags,CreationDate,Answer Date,AcceptedAnswerId,id,score,complete_text,normalized_text
0,11227809,Why is processing a sorted array faster than p...,<p>Here is a piece of C++ code that shows some...,<java><c++><performance><cpu-architecture><bra...,2012-06-27 13:51:36,2012-06-27 13:56:42,11227902,11227902,26631,why is processing a sorted array faster than p...,process sort array faster process unsort array...
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,2009-05-29 18:09:14,2009-05-29 18:13:42,927386,927386,24839,how do i undo the most recent local commits in...,undo recent local commit git accident commit w...
2,2003505,How do I delete a Git branch locally and remot...,<h4>Failed Attempts to Delete a Remote Branch:...,<git><version-control><git-branch><git-push><g...,2010-01-05 01:12:15,2010-01-05 01:13:55,2003515,2003515,19584,how do i delete a git branch locally and remot...,delet git branch local remot fail attempt dele...
3,292357,What is the difference between 'git pull' and ...,"<p>What are the differences between <a href=""h...",<git><version-control><git-pull><git-fetch>,2008-11-15 09:51:09,2008-11-15 09:52:40,292359,292359,13375,what is the difference between git pull and gi...,differ git pull git fetch differ git pull git ...
4,231767,"What does the ""yield"" keyword do?",<p>What is the use of the <code>yield</code> k...,<python><iterator><generator>,2008-10-23 22:21:11,2008-10-23 22:48:44,231855,231855,12274,what does the yield keyword do what is the use...,yield keyword use yield keyword python exampl ...


## Criando os targets

In [40]:
def split_tags(tags):
    tags = tags[1:-1]
    tags = tags.split("><")
    
    return tags

In [41]:
df['targets'] = df.Tags.apply(lambda x: split_tags(x))

In [42]:
X, y = df['complete_text'], df['targets'].apply(lambda x: x)

In [43]:
y

0         [java, c++, performance, cpu-architecture, bra...
1                  [git, version-control, git-commit, undo]
2         [git, version-control, git-branch, git-push, g...
3               [git, version-control, git-pull, git-fetch]
4                             [python, iterator, generator]
                                ...                        
105598            [unit-testing, angular, angular2-testing]
105599                               [r, vector, intervals]
105600                                      [asp.net-mvc-4]
105601                [postgresql, plpgsql, postgresql-8.4]
105602               [javascript, android, android-webview]
Name: targets, Length: 105603, dtype: object

In [57]:
df['targets'] = df.Tags.apply(lambda x: split_tags(x)[0:2])

In [58]:
df['targets']

0                     [java, c++]
1          [git, version-control]
2          [git, version-control]
3          [git, version-control]
4              [python, iterator]
                   ...           
105598    [unit-testing, angular]
105599                [r, vector]
105600            [asp.net-mvc-4]
105601      [postgresql, plpgsql]
105602      [javascript, android]
Name: targets, Length: 105603, dtype: object

In [68]:
explodedDF = df.explode('targets', ignore_index=True)

In [69]:
y = explodedDF['targets'].apply(lambda x: x)

In [78]:
words_post = explodedDF.normalized_text.apply(lambda x: str(x))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(words_post)
X = vectorizer.transform(words_post)

In [79]:
import gc

gc.collect()

548

In [81]:
print(X.shape, len(y))

(200678, 135763) 200678


In [82]:
from sklearn.model_selection import train_test_split
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
%%time
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.pipeline import make_pipeline
from sklearn.multiclass import OneVsRestClassifier

#LinearSVC(random_state=0, tol=1e-5, max_iter=10000)

clf = OneVsRestClassifier(LinearSVC(random_state=0, tol=1e-4, max_iter=1000)).fit(X_train, y_train)

In [17]:
y_pred = clf.predict_proba(X_test)

In [18]:
gc.collect()

0

In [19]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[  3   0   0 ...   0   0   0]
 [166   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]
 ...
 [  1   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]]


C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                             precision    recall  f1-score   support

                  .htaccess       0.00      1.00      0.00         3
                       .net       0.00      0.00      0.00       166
                   .net-3.5       0.00      0.00      0.00         1
                   .net-4.0       0.00      0.00      0.00         1
                     .net-5       0.00      0.00      0.00         1
              .net-assembly       0.00      0.00      0.00         1
                  .net-core       0.00      0.00      0.00         1
       abstract-syntax-tree       0.00      0.00      0.00         1
             actionscript-3       0.00      0.00      0.00         1
           active-directory       0.00      0.00      0.00         1
                      agile       0.00      0.00      0.00         3
                    airflow       0.00      0.00      0.00         2
                       ajax       0.00      0.00      0.00        12
                  alamofire      

C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
from sklearn.model_selection import cross_val_score
import numpy as np

np.mean(cross_val_score(clf, X_train, y_train, cv=5))

C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge

0.00020122685733167623